In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression

from modules import Eval

In [ ]:
origin = pd.read_csv(".\csv_data\diabetes.csv")
df_dbt = pd.DataFrame(origin)

In [ ]:
# 피처 데이터 세트 X, 레이블 데이터 세트 y를 추출.
# 맨 끝이 Outcome 컬럼으로 레이블 값임, 컬럼위치 -1 을 이용해 추출
dbt_data = df_dbt.iloc[:, 0:-1]
dbt_label = df_dbt.iloc[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dbt_data, dbt_label,
                                                    test_size = 0.2, random_state = 121, 
                                                    stratify=dbt_label)

dtree = DecisionTreeClassifier()

parameters = {"max_depth" : [5, 6, 7], "min_samples_split":[2,3]}

grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv = 3, refit = True)

grid_dtree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[["params", "mean_test_score", "rank_test_score",
            "split0_test_score", "split1_test_score", "split2_test_score"]]
print(f"GridSearchCV 최적 파라미터 : {grid_dtree.best_params_}")
print(f"GridSearchCV 최고 정확도 : {grid_dtree.best_score_ : .4f}")

estimator = grid_dtree.best_estimator_

pred = estimator.predict(X_test)
print(f"테스트 데이터 세트 정확도 {accuracy_score(y_test, pred):.4f}")

In [ ]:
# 로지스틱 회귀로 검증
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
pred_proba = lr_clf.predict_proba(X_test)[:,1]

Eval.get_clf_eval(y_test, pred)

In [ ]:
Eval.precision_recall_curve_plot(y_test, pred_proba)

In [ ]:
df_dbt.describe()

In [ ]:
plt.hist(df_dbt["Glucose"], bins=10)

In [ ]:
# 0값을 검사할 피처명 리스트 객체 설정
zero_features = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

# 전체 데이터 건수
total_count = df_dbt["Glucose"].count()

# 피처별로 반복 하면서 데이터 값이 0인 데이터 건수 추출하고, 퍼센트 계산
for feature in zero_features :
    # 전체 데이터에서, 지정된 feature column에서 0이 들어있는 row 인덱스 번호 꺼내고, 다시 거기서 그 feature column만 불러옴.
    zero_count = df_dbt[df_dbt[feature] == 0][feature].count()
    print(f"{feature} 0 건수는 {zero_count}, 비율은 {100*zero_count/total_count:.2f}%")

In [ ]:
# zero_features 리스트 내부에 저장된 개별 피처들에 대해서 0값을 평균으로 대체
df_dbt[zero_features] = df_dbt[zero_features].replace(0, df_dbt[zero_features].mean())

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df_dbt.iloc[:, :-1]
y = df_dbt.iloc[:, -1]

# StandardScaler 클래스를 이용해 피처 데이터 세트에 일괄적으로 스케일링 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2,
                                                    random_state=156, stratify=y)

# 로지스틱 회귀 적용
lr_clf_2 = LogisticRegression()
lr_clf_2.fit(X_train, y_train)
pred_2 = lr_clf_2.predict(X_test)
pred_proba_2 = lr_clf_2.predict_proba(X_test)[:, 1]

Eval.get_clf_eval(y_test, pred_2)